In [1]:
from torchvision import transforms as tf
from PIL import Image, ImageDraw, ImageFont
import numpy as np
import cv2

import glob, os
import easyocr

/home/taejukwon/anaconda3/envs/textdiffuser/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
result

[([[142, 276], [250, 276], [250, 374], [142, 374]], '상곳', 0.23758392525815616),
 ([[177, 559], [241, 559], [241, 599], [177, 599]], '나와', 0.9974574697231934),
 ([[114, 594], [296, 594], [296, 642], [114, 642]],
  '몇몇 성좌들은',
  0.9861611394302052)]

In [34]:
result[0][0]

[[142, 276], [250, 276], [250, 374], [142, 374]]

In [35]:
list

[]

In [40]:
result[1]

([[177, 559], [241, 559], [241, 599], [177, 599]], '나와', 0.9974574697231934)

In [4]:
def get_background_color(input_text):
    image = sample_image # need to be changed
    # Crop a small region around the text
    margin = 5  # pixels
    # result # need to get OCR result
    for pts, t, p in result:
        if input_text == t:
            coordinates = pts
            break
    x_min, y_min = min(coordinates, key=lambda x: x[0])[0] - margin, min(coordinates, key=lambda x: x[1])[1] - margin
    x_max, y_max = max(coordinates, key=lambda x: x[0])[0] + margin, max(coordinates, key=lambda x: x[1])[1] + margin
    cropped_region = image[y_min:y_max, x_min:x_max]

    # Calculate the most common color
    avg_color = np.mean(cropped_region, axis=(0, 1))
    r_prob = avg_color[0] / 255
    g_prob = avg_color[1] / 255
    b_prob = avg_color[2] / 255
    return r_prob, g_prob, b_prob

In [133]:
get_background_color("상곳")

(0.7227127260194725, 0.7322224683972773, 0.7438810112871262)

In [24]:
def get_OCR_prob(input_text, result):
    # result = get OCr result
    
    for pts, text, p in result:
        if input_text == text:
            output = p
    return output

In [112]:
get_OCR_prob("상곳")

0.23758392525815616

In [6]:
import pandas as pd
from Levenshtein import distance as lev_distance

# Assuming decompose_string is a custom function you have defined
from Onomato_translation import decompose_string

# Load the Korean-English Onomatopoeia CSV file


def find_most_similar_word_from_OCR(input_word):
    df = pd.read_csv('/mnt/c/Users/USER/LABis/DiffUTE/Onomatopoeia/korean_English_Onomatopoeia.csv')
    english_onomatopoeia_list = df['Korean Onomatopoeia'].tolist()
    min_distance = float('inf')
    max_distance = 0
    decomposed_input = decompose_string(input_word)

    for word in english_onomatopoeia_list:
        decomposed_word = decompose_string(word)
        distance = lev_distance(decomposed_input, decomposed_word)
        if distance < min_distance:
            min_distance = distance
            
    for word in english_onomatopoeia_list:
        decomposed_word = decompose_string(word)
        distance = lev_distance(decomposed_input, decomposed_word)
        if distance > max_distance:
            max_distance = distance

    return min_distance/max_distance

In [113]:
find_most_similar_word_from_OCR("상곳")

0.11764705882352941

In [26]:
import numpy as np
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

# Assume you have a function `get_data_for_text` that runs all your functions
# and returns a 5-dimensional vector of probabilities for each text.
def get_probs_for_text(text, result):
    r_prob, g_prob, b_prob = get_background_color(text)
    ocr_prob = get_OCR_prob(text, result)
    onomatopoeia_prob = find_most_similar_word_from_OCR(text)
    return np.array([r_prob, g_prob, b_prob, ocr_prob, onomatopoeia_prob])


In [8]:
result

[([[142, 276], [250, 276], [250, 374], [142, 374]], '상곳', 0.23758392525815616),
 ([[177, 559], [241, 559], [241, 599], [177, 599]], '나와', 0.9974574697231934),
 ([[114, 594], [296, 594], [296, 642], [114, 642]],
  '몇몇 성좌들은',
  0.9861611394302052)]

In [16]:
result[2][1]

'몇몇 성좌들은'

In [22]:

def get_OCR_result_for_d(init_img):
    os.environ['CUDA_VISIBLE_DEVICES'] = '0,'
    reader = easyocr.Reader(['ko', 'en'])
    result = reader.readtext(np.array(init_img)) # PIL to cv2 (but RGB is not changed)
    
    return result

In [20]:
list = []

for i in range(len(result)):
    list.append(result[i][1])
    print(result[i][1])

상곳
나와
몇몇 성좌들은


In [21]:
list

['상곳', '나와', '몇몇 성좌들은']

In [31]:
def concated_prob_from_image(image):
    result = get_OCR_result(image)
    
    list_t = []
    for i in range(len(result)):
        list_t.append(result[i][1])
    output = []
    for t in list_t:
        output.append(np.mean(get_probs_for_text(t, result)))
        
    return list_t, output

In [32]:
concated_prob_from_image(sample_image)

(['상곳', '나와', '몇몇 성좌들은'],
 [0.5108094379571123, 0.7495466608762762, 0.8416062418594465])